In [1]:
import numpy as np
import cmath
import math
import sympy as sy


In [2]:
g = open('./boundary_region.lmp','r')
lineb = g.readlines()
g.close()

nb = int(lineb[3].split()[0])
print(nb)
g = open('./inner_region.lmp','r')
linei = g.readlines()
g.close()
#print(len(lineb),len(linei))
ni = int(linei[3].split()[0])

number = linei[3].split()[0]
xhi = float(lineb[5].split()[1])
yhi = float(lineb[6].split()[1])
zhi = float(lineb[7].split()[1])
print(xhi,yhi,zhi)

lattice =  3.3315
a = zhi/2-lattice*0.4
b = zhi/2+lattice*0.4
c = yhi*0.4
tip = [0,c,zhi/2]
print(tip)

47800
57.703272654157 300.60769246714887 303.9193095976048
[0, 120.24307698685955, 151.9596547988024]


In [3]:
def polar(x,y,z):
    b = tip[1]
    c = tip[2]
    r = np.sqrt((y-b)**2+(z-c)**2)
    theta = np.arctan2((z-c),(y-b))
       
    return x,r,theta

In [4]:
#print(polar(0,1,np.sqrt(3)),np.sqrt(3)/2)
corff = [1,1,1,1,1]
print(np.roots(corff))

[ 0.30901699+0.95105652j  0.30901699-0.95105652j -0.80901699+0.58778525j
 -0.80901699-0.58778525j]


In [4]:
# this section tries to implement the longer displacement equations
def get_inverse(A): #this equation gives the inverse of a matrix 
    return np.linalg.inv(A)

def solve_eqn(s11,s16,s12,s66,s26,s22): # this one solve the ^4 equation and gives 4 roots
    coeff = [s11, -2*s16, 2*s12+s66, -2*s26, s22]
    a1,a2,a3,a4 = np.roots(coeff)
    #print('a1,a2,a3,a4',a1,a2,a3,a4)
    return a2, a4

def get_pq(s11,s12,s66,s22,a1,a2):
    p1 = s11*a1**2 +s12 
    p2 = s11*a2**2 +s12 
    q1 = s12*a1 +s22/a1
    q2 = s12*a2 +s22/a2
    return p1, p2, q1, q2

def displace(a1,a2,p1,p2,q1,q2,K,r,th):
    a = 1/(a1-a2)
    d = K*np.sqrt(2*r/np.pi)
    
    b1 = a1*p2*np.sqrt(np.cos(th)+a2*np.sin(th))
    c1 = a2*p1*np.sqrt(np.cos(th)+a1*np.sin(th))
    x = a*(b1-c1)*d
    
    b2 = a1*q2*np.sqrt(np.cos(th)+a2*np.sin(th))
    c2 = a2*q1*np.sqrt(np.cos(th)+a1*np.sin(th))
    y = a*(b2-c2)*d
    
    return x.real*1e10, y.real*1e10

In [5]:
# this section has the input constants
C11 = 215988.610377874 # in MPa
C22 = 215612.144829691 
C33 = 216578.15561501 

C12 = 163950.700824468
C13 = 163762.511799617
C23 = 163762.511799617

C44 = 25971.0447906336
C55 = 26116.6337858838  
C66 = 25991.3147829455 

A = np.array([[C11, C12, C13, 0, 0, 0],
              [C12, C22, C23, 0, 0, 0],
              [C13, C23, C33, 0, 0, 0],
              [0,           0,      0, C44, 0, 0],
              [0,           0,      0, 0, C55, 0],
              [0,           0,      0, 0, 0, C66]])

B = get_inverse(A)
#print(B)
sp11 = B[0][0]-B[0][2]*B[2][0]/B[2][2]
sp12 = B[0][1]-B[0][2]*B[2][1]/B[2][2]
sp66 = B[5][5]-B[5][2]*B[2][5]/B[2][2]
sp22 = B[1][1]-B[1][2]*B[2][1]/B[2][2]
sp16 = B[0][5]-B[0][2]*B[2][5]/B[2][2]
sp26 = B[1][5]-B[1][2]*B[2][5]/B[2][2]

a1,a2 = solve_eqn(sp11,sp16,sp12,sp66,sp26,sp22)
print('a1,a2',a1,a2)

p1,p2,q1,q2 = get_pq(sp11, sp12, sp66, sp22,a1,a2)

k1 = 0.5 #(MPa sqrt(m))


index = []
types = []
coor = []
for i in range(len(lineb)):
    if i>=9:
        values = lineb[i].split()
        index.append(int(values[0]))
        types.append(int(values[1]))
        coor.append([float(values[2]), float(values[3]), float(values[4])])
        #coor.append([float(values[2])*xhi, float(values[3])*Y_length, float(values[4])*Z_length])
        
new_coor = [[],[],[],[],[],[],[],[],[]]
new_y = []
new_z = []
for i in range(len(coor)):
    x,r,theta = polar(coor[i][0],coor[i][1],coor[i][2])
    d_y, d_z = displace(a1,a2,p1,p2,q1,q2,k1,r*1e-10,theta) 
    new_coor.append([x, coor[i][1]+d_y, coor[i][2]+d_z])
    new_y.append(coor[i][1]+d_y)
    new_z.append(coor[i][2]+d_z)
    
ylo = min(new_y)-1
yhi = max(new_y)+1
zlo = min(new_z)-1
zhi = max(new_z)+1



a1,a2 (-0.04747352630035637-0.9993092110014674j) (0.04747352630035567-0.9993092110014675j)


In [6]:
g = open('./boundary_displace/boundary_displace_1.lmp','w')
g.close()

g = open('./boundary_displace/boundary_displace_1.lmp','a')
count = 0
g.write('# fcc Ni oriented X=[100] Y=[010] Z=[001]. \n')
g.write('\n')
g.write(f'{nb} atoms \n')
g.write('4 atom types \n')
g.write('\n')
g.write(f'0.000000000000     {xhi}  xlo xhi \n')
g.write(f'{ylo}    {yhi}   ylo yhi \n')
g.write(f'{zlo}    {zhi}   zlo zhi \n')
g.write('\n')
g.write('Masses \n\n')
g.write('1 178.49  # Hf \n')
g.write('2 92.91  # Nb \n')
g.write('3 180.9479  # Ta \n')
g.write('4 47.867  # Ti \n\n')
g.write('Atoms #atomic \n\n')
for i in range(len(lineb)):
    if i>=9:
        count+=1
        g.write(str(index[i-9])+' '+str(types[i-9])+' '+str(new_coor[i][0])+' '+str(new_coor[i][1])+' '+str(new_coor[i][2])+'\n')
        
g.close()

print(count)

47800


In [7]:
#this is for the iner part
k1 = 0.5 #(MPa sqrt(m))

index = []
coor = []
types = []
for i in range(len(linei)):
    if i>=9:
        values = linei[i].split()
        index.append(int(values[0]))
        types.append(int(values[1]))
        coor.append([float(values[2]), float(values[3]), float(values[4])])
        #coor.append([float(values[2])*xhi, float(values[3])*Y_length, float(values[4])*Z_length])

new_coor = [[],[],[],[],[],[],[],[],[]]
new_y = []
new_z = []
for i in range(len(coor)):
    x,r,theta = polar(coor[i][0],coor[i][1],coor[i][2])
    d_y, d_z = displace(a1,a2,p1,p2,q1,q2,k1,r*1e-10,theta) 
    new_coor.append([x, coor[i][1]+d_y, coor[i][2]+d_z])
    new_y.append(coor[i][1]+d_y)
    new_z.append(coor[i][2]+d_z)

In [8]:
g = open('./inner_displace/inner_displace_1.lmp','w')
g.close()

g = open('./inner_displace/inner_displace_1.lmp','a')
count = 0
g.write('# fcc Ni oriented X=[100] Y=[010] Z=[001]. \n')
g.write('\n')
g.write(f'{ni} atoms \n')
g.write('4 atom types \n')
g.write('\n')
g.write(f'0.000000000000      {xhi}  xlo xhi \n')
g.write(f'{ylo}    {yhi}   ylo yhi \n')
g.write(f'{zlo}    {zhi}   zlo zhi \n')
g.write('\n')
g.write('Masses \n\n')
g.write('1 178.49  # Hf \n')
g.write('2 92.91  # Nb \n')
g.write('3 180.9479  # Ta \n')
g.write('4 47.867  # Ti \n\n')
g.write('Atoms #atomic \n\n')
for i in range(len(linei)):
    if i>=9:
        count+=1
        g.write(str(index[i-9])+' '+str(types[i-9])+' '+str(new_coor[i][0])+' '+str(new_coor[i][1])+' '+str(new_coor[i][2])+'\n')
        
g.close()
print(count)

239920


In [ ]:
#isotropic

In [6]:
def get_displace(r,theta,K1,v,G):
    l = K1/G
    m = np.sqrt(r/(2*np.pi))
    n = np.sin(theta/2)
    o = np.cos(theta/2)
    p = 1-2*v+(np.sin(theta/2))**2
    t = 2-2*v-(np.cos(theta/2))**2
    d_y = l*m*o*p*1e10 #*1e10 to transfer the units back to angstron
    d_z = l*m*n*t*1e10
    
    return d_y,d_z

In [32]:
#This is for the boundary, Use ISO

index = []
coor = []
for i in range(len(lineb)):
    if i>=9:
        values = lineb[i].split()
        index.append(int(values[0]))
        coor.append([float(values[2]), float(values[3]), float(values[4])])
        
new_coor = [[],[],[],[],[],[],[],[],[]]
new_y = []
new_z = []
for i in range(len(coor)):
    x,r,theta = polar(coor[i][0],coor[i][1],coor[i][2])
    k1 = 0.8 #(MPa sqrt(m))
    v = 0.37 #poisson ratio
    G = 125000#Shear modulus (MPa)
    d_y, d_z = get_displace(r*1e-10,theta,k1,v,G)  
    new_coor.append([x, coor[i][1]+d_y, coor[i][2]+d_z])
    new_y.append(coor[i][1]+d_y)
    new_z.append(coor[i][2]+d_z)
    
ylo = min(new_y)-1
yhi = max(new_y)+1
zlo = min(new_z)-1
zhi = max(new_z)+1

In [15]:
g = open('./boundary_displace.lmp','w')
g.close()

g = open('./boundary_displace.lmp','a')
count = 0
g.write('# fcc Ni oriented X=[100] Y=[010] Z=[001]. \n')
g.write('\n')
g.write('8853 atoms \n')
g.write('1 atom types \n')
g.write('\n')
g.write(f'0.000000000000     {xhi}  xlo xhi \n')
g.write(f'{ylo}    {yhi}   ylo yhi \n')
g.write(f'{zlo}    {zhi}   zlo zhi \n')
g.write('\n')
g.write('Masses \n')
g.write('\n')
g.write('1   58.69340000            # Ni \n')
g.write('\n')
g.write('Atoms # atomic \n')
g.write('\n')
for i in range(len(lineb)):
    if i>=9:
        count+=1
        g.write(f'{count} 1 '+str(new_coor[i][0])+' '+str(new_coor[i][1])+' '+str(new_coor[i][2])+'\n')
        
g.close()

print(count)

8853


In [16]:
#This is for the inner

index = []
coor = []
for i in range(len(linei)):
    if i>=9:
        values = linei[i].split()
        index.append(int(values[0]))
        coor.append([float(values[2]), float(values[3]), float(values[4])])
        
new_coor = [[],[],[],[],[],[],[],[],[]]
#new_y = []
#new_z = []
for i in range(len(coor)):
    x,r,theta = polar(coor[i][0],coor[i][1],coor[i][2])
    k1 = 0.85 #(MPa sqrt(m))
    v = 0.37 #poisson ratio
    G = 125000 #Shear modulus (MPa)
    d_y, d_z = get_displace(r*1e-10,theta,k1,v,G)  
    new_coor.append([x, coor[i][1]+d_y, coor[i][2]+d_z])
    #new_y.append(coor[i][1]+d_y)
    #new_z.append(coor[i][2]+d_z)

In [18]:
g = open('./iner_displace.lmp','w')
g.close()

g = open('./iner_displace.lmp','a')
count = 0
g.write('# fcc Ni oriented X=[100] Y=[010] Z=[001]. \n')
g.write('\n')
g.write('34347 atoms \n')
g.write('1 atom types \n')
g.write('\n')
g.write(f'0.000000000000      {xhi}  xlo xhi \n')
g.write(f'{ylo}    {yhi}   ylo yhi \n')
g.write(f'{zlo}    {zhi}   zlo zhi \n')
g.write('\n')
g.write('Masses \n')
g.write('\n')
g.write('1   58.69340000            # Ni \n')
g.write('\n')
g.write('Atoms # atomic \n')
g.write('\n')
for i in range(len(linei)):
    if i>=9:
        count+=1
        g.write(f'{count} 1 '+str(new_coor[i][0])+' '+str(new_coor[i][1])+' '+str(new_coor[i][2])+'\n')
        
g.close()
print(count)

34347


In [302]:
1/(1+247/148)

0.37468354430379747

In [55]:
3.52/2

1.76

In [10]:
2*125/(247-148)

2.525252525252525